# Fie Upon Thee, Autocorrect!

<img src="img/shakespeare.jpg" width="200">

<br><br><br>

## Recap of yesterday

I'll just leave this here so we can refer back to it.

* Quantitative data analysts distinguish between **measurements**, which are direct observations or outcomes of experiments, and **models**, which are mathematical machines that describe, predict, or explain the measurements in a quantitative way.
* Measurements can be expressed as points in an **N-dimensional space**. Since the number of measurements is finite, they can't completely fill the space.
  * Measurements can be represented in a 2-D data frame or 2-D array, in which the rows are repeated observations or experiments and the columns are observed attributes, one column/dimension per attribute.
  * Measurements can be visualized as a `scatter` plot.
  * Measurements say what _is_ true.
* Models, when questioned, provide a response for any point in the **N-dimensional space**, so a model completely fills the space.
  * Models can be represented in an N-dimensional array, as a value for each point in space, or as a function that returns a response for N arguments.
  * Models can be visualized by coloring a space with `imshow` or `contourf`, or with contour lines (like mountains on an elevation map).
  * The model-function's response may be
    * the probability that that combination of attributes exists, or
    * a prediction of some other attribute (or its probability), or
    * a category that we use to organize the data but isn't directly measurable, such as species (or its probability).
  * Models say what _would be_ true, under the given conditions, assuming that the model is accurate, etc.
* Models are algorithms involving numerical and categorical values: changing these values changes the model.
  * **Parameters** are values that we tune in an automated **fitting** procedure to find the best model for some measurements.
  * **Hyperparameters** are not part of the fitting procedure, but also impact the quality of the fitted model.
  * Models that don't accurately resemble their training data are **underfitted**.
  * Models that are too similar to their training data (take the individual points too literally—don't generalize well) are **overfitted**.
  * Both underfitting and overfitting are problematic.
* **Machine learning** is a fitting procedure, usually with very large datasets and very large numbers of parameters.
* A **neural network** is currently the most successful kind of machine learning model.
  * A neural network consists of layers of linear functions with many parameters sandwiched between non-linear functions.
  * Optimizing a neural network involves tuning the parameters of the linear functions so that the whole model fits the training data.
  * **Deep learning** is a neural network with many layers (which became feasible about 10 years ago).

<br><br><br>

## What we'll do today

Short discussion of categorical variables with the penguins.

A more detailed look at text-based data using the complete works of Shakespeare.

Build an autocomplete engine, learning a little about SQL and databases along the way.

Talk about the similarities and differences between our autocomplete engine and large language models like ChatGPT.

<br><br><br>

## Categorical variables among the penguins

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
penguins = pd.read_csv("data/penguins.csv")
penguins

<br><br><br>

In [ ]:
penguins[["species", "island", "sex"]]

In [ ]:
penguins["species"].unique()

In [ ]:
penguins["island"].unique()

In [ ]:
penguins["sex"].unique()

<br><br><br>

Many (not all!) machine learning models require inputs and outputs to be numerical. How can we do that?

<br><br><br>

### Method 1

Associate a number to each category. We've already done this.

In [ ]:
pd.Categorical(penguins["species"]).codes

In [ ]:
pd.Categorical(penguins["island"]).codes

In [ ]:
pd.Categorical(penguins["sex"]).codes

<br><br><br>

Notice that this plot is using a numerical relationship among Adelie, Gentoo, and Chinstrap to give the horizontal axis an order (Adelie first, then Gentoo, then Chinstrap).

In [ ]:
penguins["species"].value_counts().plot(kind="bar")

In [ ]:
pd.crosstab(penguins["species"], penguins["island"])

In [ ]:
fig, ax = plt.subplots()

matrix = ax.matshow(pd.crosstab(penguins["species"], penguins["island"]).values)
fig.colorbar(matrix, label="number of penguins")

ax.set_xticks([0, 1, 2], ["Biscoe", "Dream", "Torgersen"])
ax.set_yticks([0, 1, 2], ["Adelie", "Chinstrap", "Gentoo"])

None

<br><br><br>

The disadvantage of this method is that the order is not meaningful—it's something we made up—and a machine learning model might optimize for it.

It's an invitation to overfitting (which can be controlled, but still).

<br><br><br>

### Method 2

Create a dimension for each value of a categorical variable:

In [ ]:
expanded_penguins = pd.get_dummies(penguins.dropna(), columns=["species", "island", "sex"])
expanded_penguins

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

sex2D = expanded_penguins[["sex_female", "sex_male"]].values

# scatter a little, so we can see overlapping points
sex2D = sex2D.astype(np.float64) + np.random.normal(0, 0.05, (len(expanded_penguins), 2))

ax.scatter(sex2D[:, 0], sex2D[:, 1], marker=".")

ax.set_xlim(-0.3, 1.3)
ax.set_ylim(-0.3, 1.3)
ax.set_xlabel("sex_female")
ax.set_ylabel("sex_male")
ax.axhline(0, color="gray", ls=":")
ax.axvline(0, color="gray", ls=":")

None

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(projection="3d")

island3D = expanded_penguins[["island_Biscoe", "island_Dream", "island_Torgersen"]].values

# scatter a little, so we can see overlapping points
island3D = island3D.astype(np.float64) + np.random.normal(0, 0.05, (len(expanded_penguins), 3))

ax.scatter(island3D[:, 0], island3D[:, 1], island3D[:, 2], marker=".")

ax.set_xlabel("Biscoe")
ax.set_ylabel("Dream")
ax.set_zlabel("Torgersen")

None

<br><br><br>

The disadvantages of this method are that

* we quickly end up with a lot of dimensions, which uses more memory and computation time, and
* all the values between and beyond 0 and 1 are meaningless.

But if you can afford it, it's a robust way to make models!

<br><br><br>

## Dataset: the complete works of Shakespeare

This used to be a big dataset, used to illustrate large storage devices, like in [this definition of CD-ROM](https://vintageapple.org/apple_ii/pdf/Apple_IIGS__Ownwers_Guide_1986.pdf) from 1986:

<img src="img/shakespeare-a-big-dataset.png" width="600">

Now it's small enough to easily load in JupyterLite but is still big enough to be interesting.

<br><br><br>

This file comes from Project Gutenberg, [ebook #100](https://www.gutenberg.org/ebooks/100):

In [ ]:
with open("data/shakespeare.txt") as file:
    corpus = file.read()

In [ ]:
len(corpus)

In [ ]:
len(corpus) / 1e6

5.36 MB (a whole laser disk, apparently).

<br><br><br>

In [ ]:
print(corpus[100000:101000])

<br><br><br>

What distinct characters does it have?

In [ ]:
set(corpus)

<br><br><br>

## How often is "t" followed by "h"?

In [ ]:
first_character = []
next_character = []
for i in range(len(corpus) - 1):
    first_character.append(corpus[i])
    next_character.append(corpus[i + 1])

In [ ]:
first_character[100414:100439]

In [ ]:
next_character[100414:100439]

<br><br><br>

In [ ]:
pairs = pd.crosstab(first_character, next_character, rownames=["first"], colnames=["next"])
pairs

In [ ]:
sorted_by_columns = pairs[pairs.sum(axis=0).sort_values(ascending=False).index]

In [ ]:
sorted_by_both = sorted_by_columns.loc[sorted_by_columns.sum(axis=1).sort_values(ascending=False).index]

In [ ]:
sorted_by_both

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))

matrix = ax.matshow(sorted_by_both.values)

ax.set_xticks(range(len(sorted_by_both.index)), sorted_by_both.index)
ax.set_yticks(range(len(sorted_by_both.columns)), sorted_by_both.columns)

ax.set_xlim(-0.5, 25.5)
ax.set_ylim(-0.5, 25.5)
ax.set_ylabel("first character")
ax.set_xlabel("next character")
ax.xaxis.set_label_position("top")
plt.gca().invert_yaxis()

None

<br><br><br>

The bright spots are

In [ ]:
pairs.loc["e", " "]   # e followed by space (at the end of a word)

In [ ]:
pairs.loc[" ", "t"]   # space followed by t (at the beginning of a word)

In [ ]:
pairs.loc["t", "h"]   # t followed by h

In [ ]:
pairs.loc[".", " "]   # period followed by space (at the end of a sentence)

<br><br><br>

## Sequence of characters → sequence of words

We could build a per-letter autocomplete algorithm that would see "t" and suggest "h", but it wouldn't produce interesting text.

It gets more interesting if we do this at the word level.

The first step of **parsing**, an analysis of human-readable text, is **tokenizing** the input: turning raw characters into **tokens**.

Why "tokens" and not "words"? Some of our tokens will be punctuation marks, so that when your autocomplete algorithm sees `hark` it can suggest a token like `!`.

<br><br><br>

**Regular expressions** or **regex** is a mini-language for recognizing strings and parts of strings.

In [ ]:
import re

In [ ]:
#               "’" for "thou be’st" and "-" for "a-foot"       multi-digit number as a token
#                olden-time (and French) letters     🡓    "&c."  🡓  "+" matches sequences of at least 1 character
#        capital and lowercase letters       🡓       🡓     🡓     🡓  🡓   match any single character that is not (^) a space
#                                  🡓         🡓       🡓     🡓     🡓  🡓   🡓
recognize_token = re.compile("([A-Za-zÀÆÇÉàâæçèéêëîœ’-]+|&c\.|[0-9]+|[^ ])")
#                            🡑                          🡑    🡑      🡑
#              "(" and ")" form a group               "|" means "or"

In [ ]:
recognize_token.findall("Dost thou be’st 007 tokëns, &c.?")

<br><br><br>

In [ ]:
for match in recognize_token.finditer(corpus):
    token = match.group(0)
    print(repr(token))
    if token == ".":
        break

<br><br><br>

In [ ]:
tokens = []
for match in recognize_token.finditer(corpus):
    tokens.append(match.group(0))

In [ ]:
len(corpus)

In [ ]:
len(tokens)

<br><br><br>

In [ ]:
len(set(corpus))

In [ ]:
len(set(tokens))

Instead of a 103×103 table of "first", "next" pairs, this would be a 36775×36775 table. Too big!

<br><br><br>

## SQL, the language of table manipulation

Just as **regular expression** is a mini-language that we can call from Python to handle strings, **SQL** is a mini-language to deal with tables.

In [ ]:
import sqlite3

In [ ]:
db = sqlite3.connect(":memory:")
db.execute("CREATE TABLE works(title TEXT, type TEXT, characters INTEGER, year_low INTEGER, year_high INTEGER)")

<br><br><br>

(Don't take the following data too seriously; I got it from ChatGPT.)

In [ ]:
data_in_python = [
    ["The Sonnets", "poetry", None, 1609, 1609],
    ["All’s Well that Ends Well", "comedy", 23, 1604, 1605],
    ["The Tragedy of Antony and Cleopatra", "tragedy", 42, 1606, 1606],
    ["As You Like It", "comedy", 27, 1599, 1600],
    ["The Comedy of Errors", "comedy", 18, 1594, 1594],
    ["The Tragedy of Coriolanus", "tragedy", 30, 1608, 1608],
    ["Cymbeline", "mixed", 20, 1609, 1610],
    ["The Tragedy of Hamlet, Prince of Denmark", "tragedy", 30, 1599, 1601],
    ["The First Part of King Henry the Fourth", "history", 25, 1596, 1597],
    ["The Second Part of King Henry the Fourth", "history", 25, 1597, 1598],
    ["The Life of King Henry the Fifth", "history", 30, 1599, 1599],
    ["The First Part of Henry the Sixth", "history", 40, 1590, 1592],
    ["The Second Part of King Henry the Sixth", "history", 30, 1590, 1591],
    ["The Third Part of King Henry the Sixth", "history", 30, 1591, 1591],
    ["King Henry the Eighth", "history", 30, 1612, 1613],
    ["The Life and Death of King John", "history", 20, 1596, 1596],
    ["The Tragedy of Julius Caesar", "tragedy", 40, 1599, 1599],
    ["The Tragedy of King Lear", "tragedy", 20, 1605, 1606],
    ["Love’s Labour’s Lost", "comedy", 23, 1594, 1595],
    ["The Tragedy of Macbeth", "tragedy", 20, 1606, 1606],
    ["Measure for Measure", "comedy", 20, 1603, 1604],
    ["The Merchant of Venice", "comedy", 22, 1596, 1597],
    ["The Merry Wives of Windsor", "comedy", 24, 1597, 1597],
    ["A Midsummer Night’s Dream", "comedy", 21, 1595, 1596],
    ["Much Ado About Nothing", "comedy", 23, 1598, 1599],
    ["The Tragedy of Othello, the Moor of Venice", "tragedy", 21, 1603, 1604],
    ["Pericles, Prince of Tyre", "late romance", 20, 1607, 1608],
    ["King Richard the Second", "history", 20, 1595, 1595],
    ["King Richard the Third", "history", 30, 1592, 1593],
    ["The Tragedy of Romeo and Juliet", "tragedy", 20, 1595, 1595],
    ["The Taming of the Shrew", "comedy", 16, 1590, 1592],
    ["The Tempest", "late romance", 12, 1610, 1611],
    ["The Life of Timon of Athens", "tragedy", 20, 1605, 1606],
    ["The Tragedy of Titus Andronicus", "tragedy", 25, 1591, 1592],
    ["Troilus and Cressida", "mixed", 30, 1601, 1602],
    ["Twelfth Night; or, What You Will", "comedy", 18, 1601, 1602],
    ["The Two Gentlemen of Verona", "comedy", 20, 1589, 1593],
    ["The Two Noble Kinsmen", "comedy", 20, 1613, 1614],
    ["The Winter’s Tale", "comedy", 21, 1609, 1611],
    ["A Lover’s Complaint", "poetry", None, 1609, 1609],
    ["The Passionate Pilgrim", "poetry", None, 1599, 1599],
    ["The Phoenix and the Turtle", "poetry", None, 1601, 1601],
    ["The Rape of Lucrece", "poetry", 2, 1594, 1594],
    ["Venus and Adonis", "poetry", 2, 1593, 1593],
]

In [ ]:
db.executemany("INSERT INTO works VALUES(?, ?, ?, ?, ?)", data_in_python)
db.commit()

<br><br><br>

Although SQL will be more computationally efficient for what we want to do, it doesn't have a convenient way to print out and look at a table.

But we can dump it into Pandas and view it there.

`SELECT` means select columns (not rows), and `*` means "everything".

In [ ]:
pd.read_sql("SELECT * FROM works", db)

<br><br><br>

Mathematical manipulations, such as `year_uncertainty` = `year_high - year_low`.

In [ ]:
pd.read_sql("SELECT title, year_high - year_low AS year_uncertainty FROM works", db)

<br><br><br>

`WHERE` selects rows (not columns) by value.

In [ ]:
pd.read_sql("SELECT * FROM works WHERE type = 'tragedy'", db)

<br><br><br>

`GROUP BY` aggregates the data, such as counting, adding, and finding the minimum or maximum, in groups of rows with the same value of some variable (`type` in this case).

In [ ]:
pd.read_sql("SELECT type, COUNT(*) AS number, SUM(characters), MIN(year_low), MAX(year_high) FROM works GROUP BY type", db)

<br><br><br>

## Fie Upon Thee, Autocomplete!

### 2-grams

In [ ]:
ngrams = []
for i in range(len(tokens) - 1):
    ngrams.append([tokens[i], tokens[i + 1]])

In [ ]:
db.execute("CREATE TABLE ngrams2(word1 TEXT, word2 TEXT)")
db.executemany("INSERT INTO ngrams2 VALUES(?, ?)", ngrams)
db.commit()

In [ ]:
pd.read_sql("SELECT * from ngrams2", db)

<br><br><br>

In [ ]:
db.execute("CREATE TABLE ngrams2_count AS SELECT word1, word2, COUNT(*) AS count FROM ngrams2 GROUP BY word1, word2 ORDER BY -count")
db.commit()

In [ ]:
pd.read_sql("SELECT * from ngrams2_count", db)

<br><br><br>

Querying the database table as an autocomplete engine: what's the most likely token after `prompt`?

In [ ]:
prompt = ["O"]

In [ ]:
completions = db.execute("SELECT word1, word2, count FROM ngrams2_count WHERE word1=?", prompt).fetchmany(30)
completions

Okay, these are some good matches, but we want more context for larger prompts.

<br><br><br>

### 3-grams

In [ ]:
ngrams = []
for i in range(len(tokens) - 2):
    ngrams.append([tokens[i], tokens[i + 1], tokens[i + 2]])

In [ ]:
db.execute("CREATE TABLE ngrams3(word1 TEXT, word2 TEXT, word3 TEXT)")
db.executemany("INSERT INTO ngrams3 VALUES(?, ?, ?)", ngrams)
db.commit()

In [ ]:
db.execute("CREATE TABLE ngrams3_count AS SELECT word1, word2, word3, COUNT(*) AS count FROM ngrams3 GROUP BY word1, word2, word3 ORDER BY -count")
db.commit()

<br><br><br>

In [ ]:
prompt = ["O", "Romeo"]

In [ ]:
completions = db.execute("SELECT word1, word2, word3, count FROM ngrams3_count WHERE word1=? AND word2=?", prompt).fetchmany(30)
completions

<br><br><br>

If we take the first of these, where does it lead us?

In [ ]:
history = prompt + [completions[0][2]]
history

In [ ]:
completions = db.execute("SELECT word1, word2, word3, count FROM ngrams3_count WHERE word1=? AND word2=?", history[-2:]).fetchmany(30)
completions

<br><br><br>

Keep doing it! Run this cell repeatedly with control-enter.

In [ ]:
history = history + [completions[0][2]]
completions = db.execute("SELECT word1, word2, word3, count FROM ngrams3_count WHERE word1=? AND word2=?", history[-2:]).fetchmany(30)
history

<br><br><br>

Okay, maybe we shouldn't always take the most common completion. Maybe we should vary it up and randomly pick from the top 5.

In [ ]:
history = ["O", "Romeo"]

def autocomplete(history, number):
    completions = db.execute("SELECT word1, word2, word3, count FROM ngrams3_count WHERE word1=? AND word2=?", history[-2:]).fetchmany(number)
    index = np.random.randint(0, len(completions))
    return history + [completions[index][2]]

In [ ]:
history = autocomplete(history, 5)

for token in history:
    if token in ",;:—.!?“”‘&":
        prefix = ""
    else:
        prefix = " "
    print(prefix + token, end="")
print()

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

### 10-minute exercise: 4-grams

Make it better by building a database (model) of completions that are 4 tokens long!

If you need to delete a table because of a mistake, use the `DROP TABLE <name>` syntax:

In [ ]:
db.execute("DROP TABLE ngrams2")

In [ ]:
db.execute("DROP TABLE ngrams2_count")

In [ ]:
db.execute("DROP TABLE ngrams3")

In [ ]:
db.execute("DROP TABLE ngrams3_count")

<br><br><br>

After an `INSERT INTO`, you need to `db.commit`, and after a `SELECT`, you have to `fetchmany`, as in the examples above.

**Hint:** Copy the cells of the 3-gram case and modify it to make 4-grams.

Form groups of 2 or 3 and work together!

<br><br><br>

In [ ]:
ngrams = []
for i in range(len(tokens) - 3):
    ngrams.append([tokens[i], tokens[i + 1], tokens[i + 2], tokens[i + 3]])

In [ ]:
# Create an ngrams4 table and fill it with the ngrams data.
print("???")

In [ ]:
# Use a GROUP BY operation to count the number of times each unique (word1, word2, word3, word4) combination is seen.
print("???")

In [ ]:
history = ["O", "Romeo", ","]

# Extend the autocomplete function to return (word1, word2, word3, word4) rows in which (word1, word2, word3) are the last 3 in the history.
def autocomplete(history, number):
    print("???")

In [ ]:
history = autocomplete(history, 5)

for token in history:
    if token in ",;:—.!?“”‘&":
        prefix = ""
    else:
        prefix = " "
    print(prefix + token, end="")
print()

<br><br><br>

### Solution (do not peek!)

In [ ]:
ngrams = []
for i in range(len(tokens) - 3):
    ngrams.append([tokens[i], tokens[i + 1], tokens[i + 2], tokens[i + 3]])

In [ ]:
db.execute("CREATE TABLE ngrams4(word1 TEXT, word2 TEXT, word3 TEXT, word4 TEXT)")
db.executemany("INSERT INTO ngrams4 VALUES(?, ?, ?, ?)", ngrams)
db.commit()

In [ ]:
db.execute("CREATE TABLE ngrams4_count AS SELECT word1, word2, word3, word4, COUNT(*) AS count FROM ngrams4 GROUP BY word1, word2, word3, word4 ORDER BY -count")
db.commit()

In [ ]:
history = ["O", "Romeo", ","]

def autocomplete(history, number):
    completions = db.execute("SELECT word1, word2, word3, word4, count FROM ngrams4_count WHERE word1=? AND word2=? AND word3=?", history[-3:]).fetchmany(number)
    index = np.random.randint(0, len(completions))
    return history + [completions[index][3]]

In [ ]:
history = autocomplete(history, 5)

for token in history:
    if token in ",;:—.!?“”‘&":
        prefix = ""
    else:
        prefix = " "
    print(prefix + token, end="")
print()